# Preprocessing 2 - All Data 1

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler, StandardScaler, RobustScaler, MaxAbsScaler, Normalizer, FunctionTransformer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split

from sklearn import set_config
set_config(transform_output='pandas')

from category_encoders import OrdinalEncoder

from src.helpers import *

In [2]:
df = pd.read_csv('../data/train.csv')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          91 non-null     object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyle     1460 non-null   object 
 17  OverallQual    1460 non-null   int64  
 18  OverallC

# ---Pipeline---

### 1: Convert Integer to String Function Transformer

In [4]:
# Converting integers to string for the OrdinalEncoder

def convert_int_to_string(data):
  return data.astype(str)

In [5]:
# Defining the function transformer to be used in the pipeline for converting ['OverallQual', 'OverallCond'] into strings

convert_int_to_string_transformer = FunctionTransformer(convert_int_to_string)

In [6]:
preprocessor_int_to_string = ColumnTransformer([
  ('convert_int_to_string', convert_int_to_string_transformer, ['OverallQual', 'OverallCond'])
], remainder='passthrough', verbose_feature_names_out=False)

### 2. Filter for features to remove

In [7]:
# Defining the list of columns that will go into the pipeline
# Remove columns that should not be included in the pipeline

all_columns = df.columns.tolist()
columns_remove_from_filter = ['SalePrice','Id','Street','Utilities','LowQualFinSF','GarageYrBlt',
                              '3SsnPorch']

columns_to_keep = [item for item in all_columns if item not in columns_remove_from_filter]

In [8]:
# Preprocessor to filter unwanted columns, or unexpected columns.

preprocessor_filter = ColumnTransformer([
  ('passthrough','passthrough',columns_to_keep)
],remainder='drop', verbose_feature_names_out=False)

### 3. Imputing null variables

In [9]:
# Preprocessor for imputing all null variables

cols_impute_na = ['Alley', 'GarageType', 'Fence','BsmtQual', 'BsmtCond', 'BsmtExposure',
                  'BsmtFinType1','BsmtFinType2', 'FireplaceQu', 'GarageFinish', 'GarageQual','GarageCond',
                  'PoolQC', 'MiscFeature']
cols_impute_none = ['MasVnrType']
cols_impute_most_freq = ['Electrical']
cols_impute_zero = ['LotFrontage', 'MasVnrArea']

preprocessor_imputer = ColumnTransformer([
  ('impute_na', SimpleImputer(strategy='constant', fill_value='NA'), cols_impute_na),
  ('impute_none', SimpleImputer(strategy='constant', fill_value='None'), cols_impute_none),
  ('impute_zero', SimpleImputer(strategy='constant', fill_value=0), cols_impute_zero),
  ('impute_most_freq', SimpleImputer(strategy='most_frequent'), cols_impute_most_freq)
], remainder='passthrough', verbose_feature_names_out=False)

### 4. Mapping Ordinal Features

In [10]:
# Defining maps for Ordinal categories

dict_na_ex_6 = {'NA':0,'Po':1,'Fa':2,'TA':3,'Gd':4,'Ex':5}
dict_na_gd_5 = {'NA':0,'No':1,'Mn':2,'Av':3,'Gd':4}
dict_bsmt = {'NA':0,'Unf':1,'LwQ':2,'Rec':3,'BLQ':4,'ALQ':5,'GLQ':6}
dict_garage = {'NA':0,'Unf':1,'RFn':2,'Fin':3}
dict_1_10 = {1:0, 2:1, 3:2, 4:3, 5:4, 6:5, 7:6, 8:7, 9:8, 10:9}
dict_fence = {'NA':0, 'MnWw':1, 'GdWo':2, 'MnPrv':3, 'GdPrv':4}


ordinal_cat_map = [
  {'col':'BsmtQual','mapping':dict_na_ex_6},
  {'col':'BsmtCond','mapping':dict_na_ex_6},
  {'col':'BsmtExposure','mapping':dict_na_gd_5},
  {'col':'BsmtFinType1','mapping':dict_bsmt},
  {'col':'BsmtFinType2','mapping':dict_bsmt},
  {'col':'FireplaceQu','mapping':dict_na_ex_6},
  {'col':'GarageFinish','mapping':dict_garage},
  {'col':'GarageQual','mapping':dict_na_ex_6},
  {'col':'GarageCond','mapping':dict_na_ex_6},
  {'col':'PoolQC','mapping':dict_na_ex_6},
  {'col': 'OverallQual', 'mapping':dict_1_10},
  {'col': 'OverallCond', 'mapping':dict_1_10},
  {'col': 'ExterQual', 'mapping': dict_na_ex_6},
  {'col': 'ExterCond', 'mapping': dict_na_ex_6},
  {'col': 'HeatingQC', 'mapping': dict_na_ex_6},
  {'col': 'KitchenQual', 'mapping': dict_na_ex_6},
  {'col': 'Fence', 'mapping': dict_fence}
  ]

### 5. Encoding and Scaling

In [11]:
# Preprocessor for encoding and scaling

ohe_cols = ['MSSubClass','MSZoning','Alley','LotShape','LandContour','LotConfig','LandSlope',
            'Neighborhood','Condition1','Condition2','BldgType','HouseStyle','RoofStyle','RoofMatl','Exterior1st',
            'Exterior2nd','MasVnrType','Foundation','Heating', 'CentralAir','Electrical','Functional',
            'GarageType','PavedDrive','MiscFeature','MoSold','YrSold','SaleType','SaleCondition']

oe_cols = ['OverallQual','OverallCond','ExterQual','ExterCond','BsmtQual','BsmtCond','BsmtExposure',
           'BsmtFinType1','BsmtFinType2','HeatingQC','FireplaceQu', 'GarageFinish','GarageQual', 'GarageCond',
           'PoolQC','KitchenQual','Fence']

standard_cols = ['LotFrontage','GarageArea']

robust_cols = ['LotArea','MasVnrArea','BsmtFinSF1','BsmtFinSF2','BsmtUnfSF','TotalBsmtSF','1stFlrSF','2ndFlrSF',
               'GrLivArea','WoodDeckSF','OpenPorchSF','EnclosedPorch','ScreenPorch','PoolArea','MiscVal']

minmax_cols = ['YearBuilt','YearRemodAdd']


preprocessor_encode_scale = ColumnTransformer([
  ('ohe', OneHotEncoder(sparse_output=False, handle_unknown='ignore'), ohe_cols),
  ('oe', OrdinalEncoder(), oe_cols),
  ('standard_scaler', StandardScaler(), standard_cols),
  ('robust_scaler', RobustScaler(), robust_cols),
  ('minmax_scaler', MinMaxScaler(), minmax_cols)
], remainder='passthrough', verbose_feature_names_out=False)

# --- Train/Val Split, Fit & Transform ---

In [12]:
# Train/Val split, target feature is SalePrice

X = df.drop(columns='SalePrice')
y = df['SalePrice'].copy()

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [13]:
# Main pipeline, include all preprocessors

main_pipe = Pipeline([
  ('preprocessor_int_to_string', preprocessor_int_to_string),
  ('preprocessor_filter', preprocessor_filter),
  ('preprocessor_imputer', preprocessor_imputer),
  ('preprocessor_encode_scale', preprocessor_encode_scale)
])

In [14]:
# Fit and transform X_train and X_val

main_pipe.fit(X_train)

X_train_proc = main_pipe.transform(X_train)
X_val_proc = main_pipe.transform(X_val)

In [15]:
# Checking the number of columns after pipeline

len(X_train_proc.columns)

259

# --- Model Evaluation ---

In [16]:
from sklearn.linear_model import LinearRegression

lr = LinearRegression()
lr.fit(X_train_proc, y_train)

linear_results_train = evaluate_regression(lr, X_train_proc, y_train, 'baselineLR')
linear_results_val = evaluate_regression(lr, X_val_proc, y_val, 'baselineLR')

In [17]:
linear_results_train

,MAE,MSE,RMSE,MAPE,R2,adj_r2
baselineLR,14755.794674,4.758884e+08,21814.867131,0.088139,0.920214,0.897455


In [18]:
linear_results_val

,MAE,MSE,RMSE,MAPE,R2,adj_r2
baselineLR,20745.478208,1.008256e+09,31753.053091,0.124589,0.868551,-0.195364


In [19]:
val_preds = lr.predict(X_val_proc)
pred_log = np.log(val_preds)
y_log = np.log(y_val)
root_mean_squared_error(y_log, pred_log)

0.17809933645370188